# P-Center Problem

*Authors:* [Germano Barcelos](https://github.com/gegen07), [James Gaboardi](https://github.com/jGaboardi), [Levi J. Wolf](https://github.com/ljwolf), [Qunshan Zhao](https://github.com/qszhao)

This tutorial follows this guide below:
- Lattice 10x10
    - Cost matrix
    - GeoDataFrame


In [15]:
from spopt.locate import PCenter

import numpy
import geopandas
import pulp
import spaghetti
from shapely.geometry import Point

In [2]:
def simulated_geo_points(in_data, needed=20, seed=0):
    geoms = in_data.geometry
    area = tuple(in_data.total_bounds)
    simulated_points_list = []
    simulated_points_all = False
    numpy.random.seed(seed)
    while simulated_points_all == False:
        x = numpy.random.uniform(area[0], area[2], 1)
        y = numpy.random.uniform(area[1], area[3], 1)
        point = Point(x, y)
        if geoms.intersects(point)[0]:
            simulated_points_list.append(point)
        if len(simulated_points_list) == needed:
            simulated_points_all = True
    sim_pts = geopandas.GeoDataFrame(
        simulated_points_list, columns=["geometry"], crs=in_data.crs
    )

    return sim_pts

## Lattice 10x10

In [3]:
CLIENT_COUNT = 100 # quantity demand points
FACILITY_COUNT = 5 # quantity supply points

P_FACILITIES = 4

# Random seeds for reproducibility
CLIENT_SEED = 5 
FACILITY_SEED = 6 

solver = pulp.PULP_CBC_CMD() # see solvers available in pulp reference

Create lattice 10x10 with 9 vertical lines in interior.

In [4]:
lattice = spaghetti.regular_lattice((0, 0, 10, 10), 9, exterior=True)
ntw = spaghetti.Network(in_data=lattice)

Transform spaghetti instance to geopandas geodataframe.

In [5]:
gdf = spaghetti.element_as_gdf(ntw, arcs=True)

street = geopandas.GeoDataFrame(
    geopandas.GeoSeries(gdf["geometry"].buffer(0.2).unary_union),
    crs=gdf.crs,
    columns=["geometry"],
)

In [6]:
gdf

,id,geometry,comp_label
0,"(0, 1)","LINESTRING (0.00000 0.00000, 1.00000 0.00000)",0
1,"(0, 2)","LINESTRING (0.00000 0.00000, 0.00000 1.00000)",0
2,"(1, 3)","LINESTRING (1.00000 0.00000, 1.00000 1.00000)",0
3,"(1, 22)","LINESTRING (1.00000 0.00000, 2.00000 0.00000)",0
4,"(2, 3)","LINESTRING (0.00000 1.00000, 1.00000 1.00000)",0
...,...,...,...
215,"(115, 116)","LINESTRING (10.00000 5.00000, 10.00000 6.00000)",0
216,"(116, 117)","LINESTRING (10.00000 6.00000, 10.00000 7.00000)",0
217,"(117, 118)","LINESTRING (10.00000 7.00000, 10.00000 8.00000)",0
218,"(118, 119)","LINESTRING (10.00000 8.00000, 10.00000 9.00000)",0


Simulate points inside lattice bounds.

In [7]:
client_points = simulated_geo_points(street, needed=CLIENT_COUNT, seed=CLIENT_SEED)
facility_points = simulated_geo_points(
    street, needed=FACILITY_COUNT, seed=FACILITY_SEED
)

In [8]:
client_points

,geometry
0,POINT (2.10873 8.85562)
1,POINT (1.94988 9.35355)
2,POINT (4.87948 6.16214)
3,POINT (7.76544 5.19155)
4,POINT (2.88673 1.75230)
...,...
95,POINT (-0.13128 4.30248)
96,POINT (5.86437 3.42781)
97,POINT (2.20274 0.07079)
98,POINT (7.40431 10.18456)


In [9]:
facility_points

,geometry
0,POINT (9.08575 3.25259)
1,POINT (0.91963 5.98854)
2,POINT (5.31010 4.15560)
3,POINT (5.18758 5.82013)
4,POINT (6.51169 10.09833)


Simulate network service load

In [10]:
ai = numpy.random.randint(1, 12, CLIENT_COUNT)

In [11]:
ai

array([10,  7,  3,  6,  6,  2,  5,  6,  1,  3,  3,  4,  6, 10,  6,  3, 11,
        8,  9,  6,  7, 11, 11, 11,  4,  6,  2,  3,  4,  7, 10, 10,  9,  1,
       11,  4,  8,  5,  9,  2,  3,  5,  2,  6,  6,  7,  2, 10,  1,  6, 11,
        9, 10,  2,  3,  3, 10,  6, 11,  5,  2,  7,  4,  2,  5, 10,  9, 11,
       11, 11,  5,  7,  3, 10,  7,  3,  6,  3,  8,  6,  2,  2,  8,  6,  5,
        3,  9,  9,  8,  9,  1, 11,  9, 10,  3,  1,  8,  7,  1,  8])

## Using Cost Matrix

Snap points that is not spatially belong to network.

In [12]:
ntw.snapobservations(client_points, "clients", attribute=True)
ntw.snapobservations(facility_points, "facilities", attribute=True)

Calculate distance between clients and facilities.

In [13]:
cost_matrix = ntw.allneighbordistances(
    sourcepattern=ntw.pointpatterns["clients"],
    destpattern=ntw.pointpatterns["facilities"],
)

In [14]:
cost_matrix

array([[12.60302601,  3.93598651,  8.16571655,  6.04319467,  5.65607701],
       [13.10096347,  4.43392397,  8.66365401,  6.54113213,  5.15813955],
       [ 6.9095462 ,  4.2425067 ,  2.47223674,  0.34971486,  5.34955682],
       [ 2.98196832,  7.84581224,  3.45534114,  3.57786302,  6.25374871],
       [ 7.5002892 ,  6.32806975,  4.55779979,  6.43527791, 11.75939222],
       [ 0.60209077, 11.42987132,  5.03940023,  7.16192211,  9.8378078 ],
       [ 5.37335867,  6.20113923,  2.43086927,  4.30834738,  9.6324617 ],
       [ 5.40801577,  5.41976478,  3.02929369,  1.15181557,  4.85108725],
       [ 3.68807115,  8.51585171,  2.12538061,  4.24790249,  7.94717417],
       [14.22503627,  4.60274429,  9.78772681,  7.66520493,  4.98931924],
       [10.32521229,  4.99225179,  7.38272288,  9.260201  , 14.58431531],
       [ 6.65436171,  7.98732222,  5.59685112,  3.719373  ,  2.58135531],
       [11.55510375,  1.11193575,  7.11779429,  5.37988496, 10.70399927],
       [10.90832519,  1.75871431,  6.4

``PCenter.from_cost_matrix`` method creates a model aiming to minize the maximum distance between a demand point to a facility point using cost matrix calculated previously.

In [16]:
pcenter = PCenter.from_cost_matrix(cost_matrix, ai, p_facilities=P_FACILITIES)
result = pcenter.solve(solver)

Expected result is an instance of PCenter.

In [17]:
result

## Using GeoDataFrame

Assigning service load array to demand geodataframe 

In [18]:
client_points['weights'] = ai

In [19]:
client_points

,geometry,weights
0,POINT (2.10873 8.85562),10
1,POINT (1.94988 9.35355),7
2,POINT (4.87948 6.16214),3
3,POINT (7.76544 5.19155),6
4,POINT (2.88673 1.75230),6
...,...,...
95,POINT (-0.13128 4.30248),1
96,POINT (5.86437 3.42781),8
97,POINT (2.20274 0.07079),7
98,POINT (7.40431 10.18456),1


``PCenter.from_geodataframe`` method creates a model aiming to minize the maximum distance between a demand point to a facility point using geodataframes without calculating the cost matrix previously.

In [22]:
pcenter = PCenter.from_geodataframe(
    client_points, 
    facility_points, 
    "geometry", 
    "geometry", 
    "weights", 
    p_facilities=P_FACILITIES,
    distance_metric="euclidean"
)
result = pcenter.solve(solver)

Expected result is an instance of MCLP.

In [23]:
result